데이터 : data_atype.zip (수업노트에서 다운로드)

## 머신러닝
- 문제정의, 라이브러리/데이터 불러오기
- 탐색적 데이터 분석 (EDA)
- 데이터 전처리
- 피처엔지니어링
- (Train/Validation 나누기)
- 모델 선택/훈련/평가/최적화
- 예측
- (csv 생성)

## 1. 베이스라인
- 문제정의, 라이브러리 및 데이터 불러오기
- 데이터 전처리 (단순 일괄 처리)
- 모델 선택, 훈련
- 평가

## 2. 베이스라인
- 훈련/검증용 데이터 분리
- 모델 선택, 훈련
    - 의사결정나무
    - 랜덤포레스트
    - XGBoost
- 평가

## 문제1
- "<= 50K -> 0"
- "> 50K -> 1"
- 평가: 정확도

In [1]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
X_train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/X_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/X_test.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/y_train.csv")
# X_train = pd.read_csv("X_train.csv")
# X_test = pd.read_csv("X_test.csv")
# y_train = pd.read_csv("y_train.csv")

In [2]:
# 데이터 크기
X_train.shape, X_test.shape, y_train.shape

((29304, 15), (3257, 15), (29304, 2))

In [3]:
# 데이터 샘플
X_train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,State-gov,177331,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Black,Male,4386,0,40.0,United-States
1,19749,58.0,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States
2,1157,48.0,Private,125933,Some-college,10,Widowed,Exec-managerial,Unmarried,Black,Female,0,1669,38.0,United-States
3,693,58.0,Private,100313,Some-college,10,Married-civ-spouse,Protective-serv,Husband,White,Male,0,1902,40.0,United-States
4,12522,41.0,Private,195661,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,54.0,United-States


In [4]:
# 타겟 수 확인
y_train['income'].value_counts()

,count
income,
<=50K,22263
>50K,7041


In [5]:
# type확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 3.4+ MB


In [6]:
# 수치형 데이터
cols = [
        'age',
        'fnlwgt',
        'education.num',
        'capital.gain',
        'capital.loss',
        'hours.per.week'
]

In [7]:
# 수치형 데이처 통계
X_train[cols].describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [8]:
# 결측값
X_train[cols].isnull().sum()

,0
age,12
fnlwgt,0
education.num,0
capital.gain,0
capital.loss,0
hours.per.week,13


In [9]:
# 간단한 결측치 처리
# 결측치 처리 할 때 반드시!!! train 데이터 결측 할 때, test 데이터도 같이해야함!
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [10]:
# 결측치 확인
X_train[cols].isnull().sum()

,0
age,0
fnlwgt,0
education.num,0
capital.gain,0
capital.loss,0
hours.per.week,0


In [11]:
# 베이스 라인에서는 그외 전처리 및 피처 엔지니어링 생략
# 왜냐면 핵심은 머신러닝이기 떄문

In [12]:
# 머신러닝에 데이터를 넣기 위해서는 숫자로 바꿔줘야함!
# 라벨값 인 y_train 찍어보면 >50k 이런식으로 되어있어서 이것도 숫자로 바꿔줘야함!
# target값 변경
# <=50K -> 0
# >50K -> 1
# y_train['income'] == '>50k'
# 위에 코드를 통해서 이를 통해서 true 랑 false로 바꾸고 true는 0 flase는 1이니 int형으로 바꿔주면 댐

y = (y_train['income'] == '>50k').astype(int)
y[:3]

,income
0,0
1,0
2,0


In [13]:
# 데이터 확인
X_train[cols]

#여기 까지 한게 머신러닝에 데이터를 넣을 수 있는 조건을 갖추는 작업

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0
2,48.0,125933,10,0,1669,38.0
3,58.0,100313,10,0,1902,40.0
4,41.0,195661,10,0,0,54.0
...,...,...,...,...,...,...
29299,28.0,47168,6,0,0,40.0
29300,44.0,231793,16,0,0,38.0
29301,41.0,201435,9,0,0,40.0
29302,43.0,137722,9,0,0,40.0


## 머신러닝 모델

이진분류 문제임 -> 분류 문제를 할때는 Classifier 모델을 선택해 주어야함

In [14]:
# 랜덤포레스트 -> sklearn에 ensemble(앙상블) 안에 있다
# 머신러닝 할때 3가지만 기억해라 : 1. rf로 모델을 불러오고, 2.fit하고 3. preict 이렇게 하면 끝!
# 학습, 트레이닝 하고, 예측하고, 모델불러오고, 트레이닝하고 ,예측하고 이렇게 끝임
#위에 sklearn에서 제공하고 있는 머신러닝 모델들은 상기와 같은 프로세스를 따름


from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train[cols], y)  #0과 1로 변환 시킨 cols를 가져와야함
pred = rf.predict(X_test[cols])   #여기 test 도 cols로 학습 시킨게 있기 떄문에 그냥 X_test가 아닌 X_test[cols]를 넣어야함


In [15]:
# 데이터 크기 확인
len(X_test)

3257

In [16]:
# 예측 및 csv 파일 생성
# pred[:10]
# 0은 하위소득, 1은 상위 소득 이었음
pred[:10]
submit = pd.DataFrame(
         {
            'id':X_test['id'],
            'income':pred
         }
      )

submit.to_csv("00000.csv", index =False)
#제출할때 밑에 값들이 이름에 딸려오므로 index=False 반드시 해줘야함
#시험에서는 이렇게 예측한 결과 값을 데이터 프레임 형태로 만들어서 csv파일로 생성해야 되는 것

#ValueError : array length 3257 does not match index length 29304 이렇게 길이가 안맞다는 에러가뜸
# 이유는 id 를 X_test에서가 아닌 y_train 에서 가져왔기 떄문임
# 근데 왜 X_test 데이터를 가져와야하는지? 궁금함

In [17]:
# 데이터 확인(y_train)
len(pred)
len(y_train)
len(X_train)

29304

In [28]:
# 평가 (수험자는 알 수 없는 부분임) accuracy
from sklearn.metrics import accuracy_score
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/y_test.csv")
# y_test = pd.read_csv("y_test.csv")
ans = (y_test['income'] == '>50K').astype(int)
accuracy_score(ans, pred)

#위 코드 실행을 통해서 보면 0.80이상 나와야하는데 나는 0.75가 나옴 무튼 10개중 7.5개가 맞는 정확도를 가진 모델을 만든 것임

0.7543751918943813

In [19]:
y_test

,id,income
0,11574,<=50K
1,15847,<=50K
2,17655,<=50K
3,19790,>50K
4,31812,<=50K
...,...,...
3252,20589,<=50K
3253,5668,<=50K
3254,27652,<=50K
3255,14735,<=50K


## 문제2
- "<= 50K -> 0"
- "> 50K -> 1"
- 평가: roc_auc 예측 해야할 값은 : 확률

## 검증용 데이터 분리

In [40]:
#위에 평가를 시험장에서는 할수 없으니, 밑에 검증용 데이터 사용해서 제대로 됐는지 검토를 하는것!!
# 학습용 데이터와 검증용 데이터로 구분
#train_test_split이라는 명칭의 라이브러리를 쓰는것임 나누라는 함수 쓴게 아니고
from sklearn.model_selection import train_test_split
y = (y_train['income'] == '>50K').astype(int)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size = 0.1, random_state=2022)
#랜덤 스테이트값은 자유롭게 하면됨 아무거나 가능
# 4개 값이 나오게 되므로, xx, yy로 외워라
#위에 실행하면 데이터가 분리가 된것



In [41]:
# 데이터 크기
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

#아래 의사결정나무, 랜덤포레스트 ,XGBoost 다 쓸 예정 다 똑같다.

((26373, 15), (2931, 15), (26373,), (2931,))

In [45]:
y.shape

(29304,)

In [60]:
# 의사결정나무
# 위에 문제 2 보면 아까처럼 0과1이 아니라 평가는 roc_auc이고 예측은 확률이라는것을 잘 읽고 해야함
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_tr[cols], y_tr)
pred = dt.predict_proba(X_val[cols])
pred[:10]
roc_auc_score(y_val,pred[:,1])
#확률 값 예측시에는 predict_proba
# cols 쓰는 이유는 수치형만 쓸거기 때문

0.7004777509420839

In [61]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_tr[cols], y_tr)
pred = rf.predict_proba(X_val[cols])
pred[:10]
roc_auc_score(y_val,pred[:,1])

#해석 앞쪽 확률이 0이고, 뒤쪽 확률이 1임
#그래서 첫번쨰 array 보면 하위소득일확률이 97%, 상위소득일 확률 3%
#전부 보면 앞쪽 뒤쪽 합쳐서 모두 1이 나오는 것을 볼 수 있음

0.846269271618797

In [62]:
# XGBoost (부스팅 계열)
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_tr[cols], y_tr)
pred = xgb.predict_proba(X_val[cols])
pred[:1]
roc_auc_score(y_val,pred[:,1])
# 여기서 0일 확률을 구하는지, 1일 확률을 구하는지가 문제로 나옴
# 여기서는 상위소득일 확률= 1일확률을 구해볼거임
# 2개의 데이터중 하나만 선택이 되어야함
# 그리고 ROC AUC 스코으를 사용

# #밑에는 보기좋게 만드는 방법
# import pandas as pd
# # 예측 확률값을 DataFrame으로 변환하여 보기 쉽게 표시
# pred_df = pd.DataFrame(pred, columns=['Class 0 Prob', 'Class 1 Prob'])
# print(pred_df.head(10))


0.8859038904196574

In [63]:
# 평가 데이터로 예측 및 csv파일 생성
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val,pred[:,1])

# 이걸 바탕으로 의사결정,랜덤포레스트, XGboost 맨 밑에 넣어서 다 해보면 어떤 성능이 가장 좋은지 알 수 있음
#  0.70, 0.84. 0.88중 0.88이 88%정확도를 보여주므로, 제출할 때에는 XGboost로 CSV 파일을 만들어서 제출하면 됨.
# CSV 생성
pred = xgb.predict_proba(X_test[cols])
submit = pd.DataFrame(
         {
            'id':X_test['id'],
            'income':pred[:,1]
         }
      )

submit.to_csv("22222.csv", index =False)

# 평가
- 수험자는 알 수 없는 영역임

In [64]:
from sklearn.metrics import roc_auc_score
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/y_test.csv")
ans = (y_test['income'] != '<=50K').astype(int)
roc_auc_score(ans, pred[:,1])

0.880757783882784